# Static Environment

## Imports

In [ ]:
import matplotlib.pyplot as plt
from shapely.geometry import Polygon

## Default Instances

Default instances, for demonstration/visualization only.  

Available Maps: empty, corridor, obstacle, cross, swarm.  

Available num_samples: 100, 200, 300, 400, 500

In [ ]:
instances = {}

### Swarm Instance

In [ ]:
swarm_instance_dict = {
"capacity_percentage" : 0.5,
"agent_radius" : 0.1,
"map_type" : "swarm",
"num_samples" : 100, 

# config starts and goals regions
"start_regions" : [
    Polygon([[0, 40], [0, 0], [50, 0], [50, 40]]),
    Polygon([[0, 160], [0, 120], [50, 120], [50, 160]])
],
"goal_regions" : [
    Polygon([[160, 160], [160, 110], [200, 110], [200, 160]]),
    Polygon([[160, 40], [160, 0], [200, 0], [200, 40]]),
],

"num_starts" : 12,
"num_goals" : 12,
}

instances["swarm"] = swarm_instance_dict

### Obstacle Instance

In [ ]:
obstacle_instance_dict = {
"capacity_percentage" : 0.5,
"agent_radius" : 1,
"map_type" : "obstacle" ,
"num_samples" : 100 ,

"start_regions" : [
    Polygon([[0, 40], [0, 0], [50, 0], [50, 40]]),
    Polygon([[0, 160], [0, 120], [50, 120], [50, 160]])
],
"goal_regions" : [
    Polygon([[170, 160], [170, 110], [200, 110], [200, 160]]),
    Polygon([[170, 40], [170, 0], [200, 0], [200, 40]]),
],
"num_starts" : 10,
"num_goals" : 10,
}

instances["obstacle"] = obstacle_instance_dict


In [ ]:
# Curated Example
 
# num_agents = 400
# starts_idx = [88, 32, 39, 63, 17, 75, 50, 9]
# goals_idx = [28, 11, 29, 26, 54, 30, 73, 45]
# starts_agent_count = [66, 56, 71, 74, 53, 58, 62, 60]
# goals_agent_count =  [74, 52, 63, 76, 52, 62, 64, 57]


## Instance Config

In [ ]:
# Load instance
instance_config = instances["obstacle"]

## Solver Config

In [ ]:
# Set solver
solver_config = {
    "solver_name": "TEGSolver"
}

In [ ]:
from swarm_prm.utils import load_instance

solver = load_instance(instance_config, solver_config)

gaussian_prm = solver.gaussian_prm
agent_radius = instance_config["agent_radius"]
starts_idx = solver.starts_idx
goals_idx = solver.goals_idx
num_agents = solver.num_agents

### Visualize instance

In [ ]:
fig, ax = gaussian_prm.visualize_map()

for start in starts_idx:
    gaussian_prm.gaussian_nodes[start].visualize(ax)

for goal in goals_idx :
    gaussian_prm.gaussian_nodes[goal].visualize(ax, edgecolor="b")

plt.show()

## Run Solver

In [ ]:
from swarm_prm.solvers.macro import SOLVER_REGISTRY

solution = solver.solve()
assert solution["success"], "solver failed."
timestep = solution["timestep"]
paths = solution["paths"]
g_nodes = solution["g_nodes"]
candid_starts_idx = solution["starts_idx"]
candid_goals_idx = solution["goals_idx"]

num_violation, max_violation_percentage = solver.eval_capacity(solution["paths"])
print("num violation: ", num_violation)
print("max violation percentage: ", max_violation_percentage)

## APF

Parameters should be changed according to the planning instance

In [ ]:
from swarm_prm.solvers.micro.apf import APF

# apf_config = {
    # "k_attr": 0.5,
    # "k_rep": 5.2,
    # "step_size": 0.05,
    # "repulsion_radius": 0.4,
    # "goal_chisq_threshold": 5.991, # 95% CI
    # "max_rep_force": 1.0,
    # "min_dist": 0.05,
    # "max_step": 0.07,
    # "damping": 0.7
# }

apf_config = {
    "k_attr": 0.1,
    "k_rep": 500,
    "step_size": 0.5,
    "repulsion_radius": 2 * agent_radius + 0.5,
    "goal_chisq_threshold": 5.991, # 95% CI
    "max_rep_force": 10.0,
    "min_dist": 0.5,
    "max_step": 0.7,
    "damping": 0.7
}
trajectories = APF(solution["paths"], solution["g_nodes"], **apf_config).solve()

## Visualize Per-agent Paths

In [ ]:
# Static path
from matplotlib import pyplot as plt
fig, ax = gaussian_prm.visualize_map()

cmap = plt.get_cmap("rainbow")
colors = [cmap(i / num_agents) for i in range(num_agents)]

for i, path in enumerate(trajectories):
    x_coords = [loc[0] for loc in path]
    y_coords = [loc[1] for loc in path]
    ax.plot(x_coords, y_coords, '-', label='Path', color=colors[i], linewidth=0.8, alpha=0.5)

for start in starts_idx:
    gaussian_prm.gaussian_nodes[start].visualize(ax)

for goal in goals_idx:
    gaussian_prm.gaussian_nodes[goal].visualize(ax, edgecolor="b")

plt.show()

In [ ]:
# Animate path
from matplotlib.animation import FuncAnimation
from matplotlib.patches import Circle

from IPython.display import Video

speed = 50

def animate_solution(agent_radius, paths, fig, ax, fig_path="."):
    """
        Visualize solution trajectory provided instance
    """
    agents = []
    cmap = plt.get_cmap("tab10")
    
    for i in range(len(paths)):
        loc = paths[i][0]
        circle = Circle((loc[0], loc[1]), radius=agent_radius, color=cmap(i % 10))
        agents.append(circle)
        ax.add_patch(circle)

    def init():
        return agents

    def update(frame):
        frame = frame * speed
        for agent, traj in zip(agents, paths):
            agent.set_center(traj[frame])
        return agents

    anim = FuncAnimation(fig, update, frames=len(paths[0]) // speed, 
                         init_func=init, blit=True, interval=100)
    anim.save(f"{fig_path}/apf_solution.mp4", writer='ffmpeg', fps=24)
    plt.close()

fig_path = "solutions"
fig, ax = gaussian_prm.visualize_map()
animate_solution(agent_radius, trajectories, fig, ax, fig_path=fig_path)
Video(filename="solutions/apf_solution.mp4", embed=True)